# Compare performance between different FI2Pop alternatives

## Imports

In [ ]:
import json


GECCO-compatible `matplotlib` options:

In [ ]:
import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42


Import `PCGSEPy` modules:

In [ ]:
from pcgsepy.common.vecs import orientation_from_str, Vec
from pcgsepy.config import COMMON_ATOMS, HL_ATOMS, N_ITERATIONS, REQ_TILES
from pcgsepy.lsystem.rules import RuleMaker
from pcgsepy.lsystem.actions import AtomAction, Rotations
from pcgsepy.lsystem.parser import HLParser, LLParser
from pcgsepy.lsystem.solver import LSolver
from pcgsepy.lsystem.constraints import ConstraintHandler, ConstraintLevel, ConstraintTime
from pcgsepy.lsystem.constraints_funcs import components_constraint, intersection_constraint, symmetry_constraint, axis_constraint
from pcgsepy.lsystem.lsystem import LSystem
from pcgsepy.structure import block_definitions
from pcgsepy.evo.genops import expander


## Setup

In [ ]:
with open(COMMON_ATOMS, "r") as f:
    common_alphabet = json.load(f)

for k in common_alphabet:
    action, args = common_alphabet[k]["action"], common_alphabet[k]["args"]
    action = AtomAction(action)
    if action == AtomAction.MOVE:
        args = orientation_from_str[args]
    elif action == AtomAction.ROTATE:
        args = Rotations(args)
    common_alphabet[k] = {"action": action, "args": args}


In [ ]:
with open(HL_ATOMS, "r") as f:
    hl_atoms = json.load(f)

tiles_dimensions = {}
tiles_block_offset = {}
for tile in hl_atoms.keys():
    dx, dy, dz = hl_atoms[tile]["dimensions"]
    tiles_dimensions[tile] = Vec.v3i(dx, dy, dz)
    tiles_block_offset[tile] = hl_atoms[tile]["offset"]

hl_alphabet = {}
for k in common_alphabet.keys():
    hl_alphabet[k] = common_alphabet[k]

for hk in hl_atoms.keys():
    hl_alphabet[hk] = {"action": AtomAction.PLACE, "args": []}


In [ ]:
ll_alphabet = {}

for k in common_alphabet.keys():
    ll_alphabet[k] = common_alphabet[k]

# for k in block_definitions.keys():
#     if k != "":  # TODO: This is a probable bug, reported to the SE API devs
#         ll_alphabet[k] = {"action": AtomAction.PLACE, "args": [k]}


In [ ]:
used_ll_blocks = [
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerInv',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
    'MyObjectBuilder_Gyro_LargeBlockGyro',
    'MyObjectBuilder_Reactor_LargeBlockSmallGenerator',
    'MyObjectBuilder_CargoContainer_LargeBlockSmallContainer',
    'MyObjectBuilder_Cockpit_OpenCockpitLarge',
    'MyObjectBuilder_Thrust_LargeBlockSmallThrust',
    'MyObjectBuilder_InteriorLight_SmallLight',
    'MyObjectBuilder_CubeBlock_Window1x1Slope',
    'MyObjectBuilder_CubeBlock_Window1x1Flat',
    'MyObjectBuilder_InteriorLight_LargeBlockLight_1corner'
]

for k in used_ll_blocks:
    ll_alphabet[k] = {"action": AtomAction.PLACE, "args": [k]}


## L-System components

In [ ]:
hl_rules = RuleMaker(ruleset='hlrules').get_rules()
ll_rules = RuleMaker(ruleset='llrules').get_rules()

hl_parser = HLParser(rules=hl_rules)
ll_parser = LLParser(rules=ll_rules)

hl_solver = LSolver(parser=hl_parser,
                    atoms_alphabet=hl_alphabet,
                    extra_args={
                        'tiles_dimensions': tiles_dimensions,
                        'tiles_block_offset': tiles_block_offset,
                        'll_rules': ll_rules
                    })
ll_solver = LSolver(parser=ll_parser,
                    atoms_alphabet=dict(hl_alphabet, **ll_alphabet),
                    extra_args={})


In [ ]:
rcc1 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc1.extra_args["req_tiles"] = ['cockpit']

rcc2 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc2.extra_args["req_tiles"] = [
    'corridorcargo', 'corridorgyros', 'corridorreactors']

rcc3 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc3.extra_args["req_tiles"] = ['thrusters']

nic = ConstraintHandler(
    name="no_intersections",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.DURING,
    f=intersection_constraint,
    extra_args={
        'alphabet': dict(hl_alphabet, **ll_alphabet)
    },
    needs_ll=True
)
nic.extra_args["tiles_dimensions"] = tiles_dimensions

sc = ConstraintHandler(
    name="symmetry",
    level=ConstraintLevel.SOFT_CONSTRAINT,
    when=ConstraintTime.END,
    f=symmetry_constraint,
    extra_args={
        'alphabet': dict(hl_alphabet, **ll_alphabet)
    }
)


In [ ]:
lsystem = LSystem(
    hl_solver=hl_solver, ll_solver=ll_solver, names=[
        'HeadModule', 'BodyModule', 'TailModule']
)


In [ ]:
lsystem.add_hl_constraints(cs=[
    [nic, rcc1],
    [nic, rcc2],
    [nic, rcc3]
])

lsystem.add_ll_constraints(cs=[
    [sc],
    [sc],
    [sc]
])


In [ ]:
expander.initialize(rules=lsystem.hl_solver.parser.rules)


In [ ]:
from pcgsepy.evo.fitness import *

feasible_fitnesses = [
    #     Fitness(name='BoundingBox',
    #             f=bounding_box_fitness,
    #             bounds=(0, 1)),
    Fitness(name='BoxFilling',
            f=box_filling_fitness,
            bounds=(0, 1)),
    Fitness(name='FuncionalBlocks',
            f=func_blocks_fitness,
            bounds=(0, 1)),
    Fitness(name='MajorMediumProportions',
            f=mame_fitness,
            bounds=(0, 1)),
    Fitness(name='MajorMinimumProportions',
            f=mami_fitness,
            bounds=(0, 1))
]

## FI2Pop

In [ ]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from sklearn.exceptions import NotFittedError

class NNEstimator(nn.Module):
    def __init__(self):
        super(NNEstimator, self).__init__()
        self.l1 = nn.Linear(3, 16)
        self.l2 = nn.Linear(16, 32)
        self.l3 = nn.Linear(32, 3)

        self.criterion = nn.MSELoss()
        # self.optim = th.optim.SGD(params=self.parameters(),
        #                           lr=1e-3,
        #                           momentum=0.5)
        self.optim = th.optim.Adam(params=self.parameters())
        
        self.is_trained = False
    
    def forward(self, x):
        out = F.relu(self.l1(x))
        out = F.relu(self.l2(out))
        out = F.hardsigmoid(self.l3(out))
        return out
    
    def fit(self, xs, ys):
        self.train()
        for epoch in range(20):
            for x, y in zip(xs, ys):
                x = th.tensor(x, dtype=th.float32).reshape((1, -1))
                y = th.tensor(y, dtype=th.float32).reshape((1, -1))
                self.optim.zero_grad()
                output = self.forward(x)
                loss = self.criterion(output, y)
                loss.backward()
                self.optim.step()
                
                if epoch == 9:
                    print(f'NN loss is {loss.item()}')
        self.is_trained = True
    
    def predict(self, x):
        if not self.is_trained:
            raise NotFittedError('Train the NN first')
        self.eval()
        x = th.tensor(x, dtype=th.float32)
        return self.forward(x).detach().numpy()
            

In [ ]:
from typing import Any, Dict, List, Tuple
from tqdm.notebook import trange
import numpy as np
from pcgsepy.evo.fitness import Fitness
from pcgsepy.lsystem.solution import CandidateSolution
from pcgsepy.config import POP_SIZE, N_RETRIES, N_GENS
from pcgsepy.fi2pop.fi2pop import create_new_pool, subdivide_solutions
from pcgsepy.fi2pop.utils import create_new_pool, reduce_population

from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.exceptions import NotFittedError

from pcgsepy.fi2pop.utils import DimensionalityReducer

class FI2PopSolver:
    def __init__(self,
                 feasible_fitnesses: List[Fitness],
                 lsystem: LSystem):
        """Create the FI2Pop solver.

        Args:
            feasible_fitnesses (List[Fitness]): The list of fitnesses.
            lsystem (LSystem): The L-system object.
        """
        self.feasible_fitnesses = feasible_fitnesses
        self.lsystem = lsystem
        self.ftop = []
        self.itop = []
        self.fmean = []
        self.imean = []

        self.ffs, self.ifs = [], []

        # number of total soft constraints
        self.nsc = [c for c in self.lsystem.all_hl_constraints if c.level == ConstraintLevel.SOFT_CONSTRAINT]
        self.nsc = [c for c in self.lsystem.all_ll_constraints if c.level == ConstraintLevel.SOFT_CONSTRAINT]
        self.nsc = len(self.nsc)
        
        self.buffer = {
            'X': [],
            'Y': []
        }
        # self.estimator = LinearRegression()
        # self.estimator = NNEstimator()
        # self.estimator = KernelRidge()
        self.estimator = GaussianProcessRegressor(kernel=DotProduct() + WhiteKernel(),
                                                  random_state=0)
        
        # self.reducer = DimensionalityReducer(n_components=10,
        #                                      max_dims=500)

    def _compute_fitness(self,
                         cs: CandidateSolution,
                         extra_args: Dict[str, Any]) -> float:
        """Compute the fitness of a single candidate solution.

        Args:
            cs (CandidateSolution): The candidate solution.
            extra_args (Dict[str, Any]): Additional arguments used in the fitness function.

        Returns:
            float: The fitness value.
        """
        return [f(cs, extra_args) for f in self.feasible_fitnesses]

    def _generate_initial_populations(self,
                                      pops_size: int = POP_SIZE,
                                      n_retries: int = N_RETRIES) -> Tuple[List[CandidateSolution], List[CandidateSolution]]:
        """Generate the initial populations.

        Args:
            pops_size (int, optional): The size of the population. Defaults to POP_SIZE.
            n_retries (int, optional): The number of retries. Defaults to N_RETRIES.

        Returns:
            Tuple[List[CandidateSolution], List[CandidateSolution]]: The Feasible and Infeasible populations.
        """
        feasible_pop, infeasible_pop = [], []
        self.lsystem.disable_sat_check()
        with trange(n_retries, desc='Initialization ') as iterations:
            for i in iterations:
                solutions = self.lsystem.apply_rules(starting_strings=['head', 'body', 'tail'],
                                                     iterations=[1, N_ITERATIONS, 1],
                                                     create_structures=False,
                                                     make_graph=False)
                subdivide_solutions(lcs=solutions,
                                    lsystem=self.lsystem)
                for cs in solutions:
                    cs.fitness = self._compute_fitness(cs=cs,
                                                       extra_args={
                                                           'alphabet': self.lsystem.ll_solver.atoms_alphabet
                                                           })
                    if cs.is_feasible and len(feasible_pop) < pops_size and cs not in feasible_pop:
                        cs.c_fitness = sum(cs.fitness) + (self.nsc - cs.ncv)
                        feasible_pop.append(cs)
                    elif not cs.is_feasible and len(infeasible_pop) < pops_size and cs not in feasible_pop:
                        # cs.c_fitness = np.clip(np.abs(np.random.normal(loc=0, scale=1)), 0, 1)
                        cs.c_fitness = sum(cs.fitness)
                        infeasible_pop.append(cs)
                iterations.set_postfix(ordered_dict={'fpop-size': f'{len(feasible_pop)}/{pops_size}',
                                                     'ipop-size': f'{len(infeasible_pop)}/{pops_size}'},
                                       refresh=True)
                if i == n_retries or (len(feasible_pop) == pops_size and len(infeasible_pop) == pops_size):
                    break
        
        # xs = [x._content for x in feasible_pop] + [x._content for x in infeasible_pop]
        # self.reducer.fit(xs)
        
        return feasible_pop, infeasible_pop

    def initialize(self) -> Tuple[List[CandidateSolution], List[CandidateSolution]]:
        """Initialize the solver by generating the initial populations.

        Returns:
            Tuple[List[CandidateSolution], List[CandidateSolution]]: The Feasible and Infeasible populations.
        """
        f_pop, i_pop = self._generate_initial_populations()
        f_fitnesses = [cs.c_fitness for cs in f_pop]
        i_fitnesses = [cs.c_fitness for cs in i_pop]
        self.ftop.append(max(f_fitnesses))
        self.fmean.append(sum(f_fitnesses) / len(f_fitnesses))
        self.itop.append(max(i_fitnesses))
        self.imean.append(sum(i_fitnesses) / len(i_fitnesses))
        self.ffs.append([self.ftop[-1], self.fmean[-1]])
        self.ifs.append([self.itop[-1], self.imean[-1]])
        print(f'Created Feasible population of size {len(f_pop)}: t:{self.ftop[-1]};m:{self.fmean[-1]}')
        print(f'Created Infeasible population of size {len(i_pop)}: t:{self.itop[-1]};m:{self.imean[-1]}')
        return f_pop, i_pop

    def fi2pop(self,
               f_pop: List[CandidateSolution],
               i_pop: List[CandidateSolution],
               n_iter: int = N_GENS) -> Tuple[List[CandidateSolution], List[CandidateSolution]]:
        """Apply the FI2Pop algorithm to the given populations for `n_iter` steps.

        Args:
            f_pop (List[CandidateSolution]): The Feasible population.
            i_pop (List[CandidateSolution]): The Infeasible population.
            n_iter (int, optional): The number of iterations to run for. Defaults to N_GENS.

        Returns:
            Tuple[List[CandidateSolution], List[CandidateSolution]]: The Feasible and the Infeasible populations.
        """
        f_pool = []
        i_pool = []
        with trange(n_iter, desc='Generation ') as gens:
            for gen in gens:
                # place the infeasible population in the infeasible pool
                i_pool.extend(i_pop)
                
                f_pool.extend(f_pop)
                
                # create offsprings from feasible population
                new_pool = create_new_pool(population=f_pop,
                                           generation=gen)
                # if feasible, add to feasible pool
                # if infeasible, add to infeasible pool
                subdivide_solutions(lcs=new_pool,
                                    lsystem=self.lsystem)
                for cs in new_pool:
                    cs.ll_string = self.lsystem.hl_to_ll(cs=cs).string
                    cs.fitness = self._compute_fitness(cs=cs,
                                                       extra_args={
                                                           'alphabet': self.lsystem.ll_solver.atoms_alphabet
                                                           })
                    if cs.is_feasible:
                        if cs not in f_pool:
                            cs.c_fitness = sum(cs.fitness) + (self.nsc - cs.ncv)
                            f_pool.append(cs)
                    else:
                        if cs not in i_pool:
                            try:
                                # red = self.reducer.reduce_dims(cs._content)
                                cs.c_fitness = np.sum(self.estimator.predict(np.asarray(cs.fitness).reshape(1, -1)))
                                # cs.c_fitness = np.sum(self.estimator.predict(np.asarray(red).reshape(1, -1)))
                            except NotFittedError:
                                cs.c_fitness = sum(cs.fitness)
                            i_pool.append(cs)
                i_pool = list(set(i_pool))
                # reduce the infeasible pool if > pops_size
                if len(i_pool) > POP_SIZE:
                    i_pool = reduce_population(population=i_pool,
                                               to=POP_SIZE,
                                               minimize=False)
                # set the infeasible pool as the infeasible population
                i_pop = [e for e in i_pool]
                # create offsprings from infeasible population
                new_pool = create_new_pool(population=i_pop,
                                           generation=gen,
                                           minimize=False)
                # if feasible, add to feasible pool
                # if infeasible, add to infeasible pool
                subdivide_solutions(lcs=new_pool,
                                    lsystem=self.lsystem)
                for cs in new_pool:
                    cs.ll_string = self.lsystem.hl_to_ll(cs=cs).string
                    cs.fitness = self._compute_fitness(cs=cs,
                                                       extra_args={
                                                           'alphabet': self.lsystem.ll_solver.atoms_alphabet
                                                           })
                    if cs.is_feasible:
                        if cs not in f_pool:
                            cs.c_fitness = sum(cs.fitness) + (self.nsc - cs.ncv)
                            
                            for p in cs.parents:
                                if not p.is_feasible:
                                    if p.fitness not in self.buffer['X']:
                                    # red = self.reducer.reduce_dims(p._content)
                                    # if red not in self.buffer['X']:
                                        self.buffer['X'].append(p.fitness)
                                        # self.buffer['X'].append(red)
                                        self.buffer['Y'].append(cs.fitness)
                                    else:
                                        idx = self.buffer['X'].index(p.fitness)
                                        # idx = self.buffer['X'].index(red)
                                        curr_y = self.buffer['Y'][idx]
                                        for i, (y1, y2) in enumerate(zip(curr_y, cs.fitness)):
                                            curr_y[i] = (y1 + y2) / 2  
                            
                            f_pool.append(cs)                                      
                        
                    else:
                        if cs not in i_pool:
                            try:
                                # red = self.reducer.reduce_dims(cs._content)
                                cs.c_fitness = np.sum(self.estimator.predict(np.asarray(cs.fitness).reshape(1, -1)))
                                # cs.c_fitness = np.sum(self.estimator.predict(np.asarray(red).reshape(1, -1)))
                            except NotFittedError:
                                cs.c_fitness = sum(cs.fitness)  # cs.ncv
                            i_pool.append(cs)
                
                if len(self.buffer['X']) > 0:
                    self.estimator.fit(np.asarray(self.buffer['X']),
                                       np.asarray(self.buffer['Y']))
                
                # reduce the feasible pool if > pops_size
                f_pool = list(set(f_pool))
                if len(f_pool) > POP_SIZE:
                    f_pool = reduce_population(population=f_pool,
                                               to=POP_SIZE)
                # set the feasible pool as the feasible population
                f_pop = [e for e in f_pool]
                # update tracking
                f_fitnesses = [cs.c_fitness for cs in f_pop]
                i_fitnesses = [cs.c_fitness for cs in i_pop]
                self.ftop.append(max(f_fitnesses))
                self.fmean.append(sum(f_fitnesses) / len(f_fitnesses))
                self.itop.append(max(i_fitnesses))
                self.imean.append(sum(i_fitnesses) / len(i_fitnesses))
                self.ffs.append([self.ftop[-1], self.fmean[-1]])
                self.ifs.append([self.itop[-1], self.imean[-1]])
                gens.set_postfix(ordered_dict={'top-f': self.ftop[-1],
                                               'mean-f': self.fmean[-1],
                                               'top-i': self.itop[-1],
                                               'mean-i': self.imean[-1],
                                               'samples': len(self.buffer['X']),
                                               'len_f': len(f_pop),
                                               'len_i': len(i_pop)},
                                 refresh=True)
                
                # print('## FEASIBLE ##\n', f_pop)
                # print('## INFEASIBLE ##\n', i_pop)

        return f_pop, i_pop

## Experiments

In [ ]:
from pcgsepy.config import N_RUNS, EXP_NAME

f_fitnesses_hist = []
i_fitnesses_hist = []

run_experiment = True

In [ ]:
import numpy as np
from pcgsepy.lsystem.solution import CandidateSolution


def save_stats(cs: CandidateSolution) -> None:
    with open(f'{EXP_NAME}_atoms.log', 'a') as f:
        f.write(f'\n\n{cs.string}')
        f.write(f'\nFitness: {cs.fitness}')

In [ ]:
if run_experiment:
    with trange(N_RUNS, desc='Running experiments') as iterations:
        for n in iterations:
            solver = FI2PopSolver(feasible_fitnesses=feasible_fitnesses,
                                    lsystem=lsystem)

            f_pop, i_pop = solver.initialize()

            f_pop, i_pop = solver.fi2pop(f_pop=f_pop,
                                         i_pop=i_pop,
                                         n_iter=N_GENS)

            f_fitnesses_hist.append(solver.ffs)
            i_fitnesses_hist.append(solver.ifs)

            f_fitnesses = [cs.c_fitness for cs in f_pop]
            i_fitnesses = [cs.c_fitness for cs in f_pop]
            
            save_stats(cs=f_pop[f_fitnesses.index(max(f_fitnesses))])

            iterations.set_postfix(ordered_dict={'f_fit': np.max(f_fitnesses),
                                                 'i_fit': np.max(i_fitnesses)},
                                   refresh=True)

In [ ]:
# run_experiment = False
# EXP_NAME = 'pca_nn'

In [ ]:
if run_experiment:
    from pcgsepy.config import POP_SIZE

    ffs = np.empty(shape=(N_RUNS, 1 + N_GENS, 2))
    for r, rv in enumerate(f_fitnesses_hist):
        for g, gv in enumerate(rv):
            ffs[r, g, :] = f_fitnesses_hist[r][g][:]
    ifs = np.empty(shape=(N_RUNS, 1 + N_GENS, 2))
    for r, rv in enumerate(i_fitnesses_hist):
        for g, gv in enumerate(rv):
            ifs[r, g, :] = i_fitnesses_hist[r][g][:]

    with open(f'results/{EXP_NAME}_metrics.npz', 'wb') as f:
        np.savez(f, ffs, ifs)
else:
    with open(f'results/{EXP_NAME}_metrics.npz', 'rb') as f:
        npzfile = np.load(f)
        ffs = npzfile['arr_0']
        ifs = npzfile['arr_1']

In [ ]:
import matplotlib.pyplot as plt

SMALL_SIZE = 20
MEDIUM_SIZE = 22
BIGGER_SIZE = 26

PAD_TITLE_SIZE = 20
PAD_LABEL_SIZE = 10

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
import matplotlib.pyplot as plt

ftfs = np.max(ffs, axis=2)
fmfs = np.mean(ffs, axis=2)

ftm = np.mean(ftfs, axis=0)
fts = np.std(ftfs, axis=0)

fmm = np.mean(fmfs, axis=0)
fms = np.std(fmfs, axis=0)

plt.grid()
plt.plot(range(len(ftm)), ftm, label=f'Top fitness', c='blue', lw=2)
plt.fill_between(range(len(fts)), (ftm - (.5 * fts)), (ftm + (.5 * fts)), color='blue', alpha=0.2)
plt.fill_between(range(len(fts)), (ftm - fts), (ftm + fts), color='blue', alpha=0.1)
plt.plot(range(len(fmm)), fmm, label=f'Mean fitness', c='darkgreen', lw=2)
plt.fill_between(range(len(fms)), (fmm - (.5 * fms)), (fmm + (.5 * fms)), color='darkgreen', alpha=0.2)
plt.fill_between(range(len(fms)), (fmm - fms), (fmm + fms), color='darkgreen', alpha=0.1)
# plt.ylim(0, 5.5)
# plt.xlim(N_GENS+1)
plt.legend(loc='lower right')
plt.title(f'Avg. FPop fitness ({N_RUNS} runs) {EXP_NAME}', pad=PAD_TITLE_SIZE)
plt.ylabel('Fitness', labelpad=PAD_LABEL_SIZE)
plt.xlabel('Generations', labelpad=PAD_LABEL_SIZE)
plt.autoscale(enable=True, axis='x', tight=True)
plt.savefig(f'results/{EXP_NAME}-fpop-avgf.png', transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

itfs = np.max(ifs, axis=2)
imfs = np.mean(ifs, axis=2)

itm = np.mean(itfs, axis=0)
its = np.std(itfs, axis=0)

imm = np.mean(imfs, axis=0)
ims = np.std(imfs, axis=0)

plt.grid()
plt.plot(range(len(itm)), itm, label=f'Top fitness', c='blue', lw=2)
plt.fill_between(range(len(its)), (itm - (.5 * its)), (itm + (.5 * its)), color='blue', alpha=0.2)
plt.fill_between(range(len(its)), (itm - its), (itm + its), color='blue', alpha=0.1)
plt.plot(range(len(imm)), imm, label=f'Mean fitness', c='darkgreen', lw=2)
plt.fill_between(range(len(ims)), (imm - (.5 * ims)), (imm + (.5 * ims)), color='darkgreen', alpha=0.2)
plt.fill_between(range(len(ims)), (imm - ims), (imm + ims), color='darkgreen', alpha=0.1)
# plt.ylim(len(fitnesses))
# plt.xlim(N_GENS+1)
plt.legend(loc='lower right')
plt.title(f'Avg. IPop fitness ({N_RUNS} runs) {EXP_NAME}', pad=PAD_TITLE_SIZE)
plt.ylabel('Fitness', labelpad=PAD_LABEL_SIZE)
plt.xlabel('Generations', labelpad=PAD_LABEL_SIZE)
plt.autoscale(enable=True, axis='x', tight=True)
plt.savefig(f'results/{EXP_NAME}-ipop-avgf.png', transparent=True, bbox_inches='tight')
plt.show()